In [22]:
from google.colab import userdata #userdata → A Colab utility that allows secure access to stored secrets (like API keys)
import os # to intreact with our operating system

os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
#In this we taske AWS Access Key ID stored in Colab and expose it to the system by setting it as environment variable
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = userdata.get("AWS_DEFAULT_REGION")

In [23]:
!pip install boto3

In [ ]:
import boto3 #boto3 library help us to communicate with AWS services, here we are using S3 hence we have imported it.

# Colab automatically injects secrets as env vars
session = boto3.Session(
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    region_name=os.environ['AWS_DEFAULT_REGION']
)

s3 = session.client('s3')
#This code creates a boto3 session using AWS credentials stored in environment variables and then initializes an S3 client to interact with Amazon S3 services securely.

In [ ]:
s3.list_objects_v2(Bucket='29-01-data-analysis-ipl')
#This line fetches a list of objects (files) stored inside the specified S3 bucket.
#list_objects_v2 : Returns metadata about objects inside a bucket.

{'ResponseMetadata': {'RequestId': 'AVXXQ7MT7J4TND3Z',
  'HostId': 'zopulY2AkQqsQuK/fq697Q9EJeq8PM2OZPg79YcFeWqSVrTf2YSyb2029BNfm6xAbHEkoXqyCNPnPVx+N5SsFENukYrxPv/o',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'zopulY2AkQqsQuK/fq697Q9EJeq8PM2OZPg79YcFeWqSVrTf2YSyb2029BNfm6xAbHEkoXqyCNPnPVx+N5SsFENukYrxPv/o',
   'x-amz-request-id': 'AVXXQ7MT7J4TND3Z',
   'date': 'Thu, 19 Feb 2026 18:46:20 GMT',
   'x-amz-bucket-region': 'ap-south-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'Ball_By_Ball.csv',
   'LastModified': datetime.datetime(2026, 1, 29, 7, 20, 26, tzinfo=tzlocal()),
   'ETag': '"c4d5eef96f01a9cb3a0406cb1d932a02-2"',
   'ChecksumAlgorithm': ['CRC64NVME'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 25099132,
   'StorageClass': 'STANDARD'},
  {'Key': 'Match.csv',
   'LastModified': datetime.datetime(2026, 1, 29, 7, 20, 27, tzinfo=tzlocal()),
  

In [ ]:
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ColabPySpark") \
    .getOrCreate()

spark

**Loading Data**

In [ ]:
import pandas as pd

obj1 = s3.get_object(
    Bucket="29-01-data-analysis-ipl",
    Key = "Ball_By_Ball.csv"
)

df1 = pd.read_csv(obj1["Body"])

In [ ]:
obj2 = s3.get_object(
    Bucket="29-01-data-analysis-ipl",
    Key = "Match.csv"
)

df2 = pd.read_csv(obj2["Body"])

In [ ]:
import io
import pandas as pd

obj3 = s3.get_object(
    Bucket="29-01-data-analysis-ipl",
    Key = "Player.csv"
)

# Read the body content and decode it explicitly using 'cp1252'
body_content = obj3["Body"].read().decode('cp1252')

# Use io.StringIO to create a file-like object from the decoded string
df3 = pd.read_csv(io.StringIO(body_content))

In [ ]:
obj4 = s3.get_object(
    Bucket="29-01-data-analysis-ipl",
    Key = "Player_match.csv"
)

# Read the body content and decode it explicitly using 'cp1252'
body_content = obj4["Body"].read().decode('cp1252')

# Use io.StringIO to create a file-like object from the decoded string
df4 = pd.read_csv(io.StringIO(body_content))


In [ ]:
obj5 = s3.get_object(
    Bucket="29-01-data-analysis-ipl",
    Key = "Team.csv"
)

df5 = pd.read_csv(obj5["Body"])


**Data Preprocessing**

In [24]:
ball_by_ball_df = spark.createDataFrame(df1)
ball_by_ball_df.show(5)

+--------+-------+-------+----------+------------+------------+------------------------+----------+-----------+----------+-----+-------+----+-------+-------+-------------+--------------+------+------+-------+---+------------+-------+-----------------+----------+----------------+-------------+----------+------+-------+-----------+------+----------+--------+----------------+---------+-------------------+-------------+----------------+----------+---------------+---------+------------------+--------------+--------------+------------+-------------+-----------+---------+---------+----------------+-------------+---------+---------+----------------+-------------+-------+-------+----------------+-----------+
|MatcH_id|Over_id|Ball_id|Innings_No|Team_Batting|Team_Bowling|Striker_Batting_Position|Extra_Type|Runs_Scored|Extra_runs|Wides|Legbyes|Byes|Noballs|Penalty|Bowler_Extras|      Out_type|Caught|Bowled|Run_out|LBW|Retired_hurt|Stumped|caught_and_bowled|hit_wicket|ObstructingFeild|Bowler_Wicke

In [25]:
match_df = spark.createDataFrame(df2)
match_df.show(5)

+--------+--------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+--------------------+--------------------+---------+--------+------------+-----------+----------+----------+
|Match_SK|match_id|               Team1|               Team2|match_date|Season_Year|          Venue_Name| City_Name|Country_Name|         Toss_Winner|        match_winner|Toss_Name|Win_Type|Outcome_Type|  ManOfMach|Win_Margin|Country_id|
+--------+--------+--------------------+--------------------+----------+-----------+--------------------+----------+------------+--------------------+--------------------+---------+--------+------------+-----------+----------+----------+
|       0|  335987|Royal Challengers...|Kolkata Knight Ri...| 4/18/2008|       2008|M Chinnaswamy Sta...| Bangalore|       India|Royal Challengers...|Kolkata Knight Ri...|    field|    runs|      Result|BB McCullum|     140.0|         1|
|       1|  335988|     Kings XI Punjab| Chennai

In [26]:
player_df = spark.createDataFrame(df3)
player_df.show(5)

+---------+---------+---------------+----------+--------------+------------------+------------+
|PLAYER_SK|Player_Id|    Player_Name|       DOB|  Batting_hand|     Bowling_skill|Country_Name|
+---------+---------+---------------+----------+--------------+------------------+------------+
|        0|        1|     SC Ganguly|  7/8/1972| Left-hand bat|  Right-arm medium|       India|
|        1|        2|    BB McCullum| 9/27/1981|Right-hand bat|  Right-arm medium| New Zealand|
|        2|        3|     RT Ponting|12/19/1974|Right-hand bat|  Right-arm medium|   Australia|
|        3|        4|      DJ Hussey| 7/15/1977|Right-hand bat|Right-arm offbreak|   Australia|
|        4|        5|Mohammad Hafeez|10/17/1980|Right-hand bat|Right-arm offbreak|    Pakistan|
+---------+---------+---------------+----------+--------------+------------------+------------+
only showing top 5 rows


In [27]:
player_match_df = spark.createDataFrame(df4)
player_match_df.show(5)

+---------------+---------------+--------+---------+-----------+----------+--------------+--------------------+------------+---------+--------------------+--------------------+-----------+----------------+---------------+------------------+--------------+--------------+--------------+---------------+-------------+--------------+
|Player_match_SK|PlayerMatch_key|Match_Id|Player_Id|Player_Name|       DOB|  Batting_hand|       Bowling_skill|Country_Name|Role_Desc|         Player_team|        Opposit_Team|Season_year|is_manofThematch|Age_As_on_match|IsPlayers_Team_won|Batting_Status|Bowling_Status|Player_Captain|Opposit_captain|Player_keeper|Opposit_keeper|
+---------------+---------------+--------+---------+-----------+----------+--------------+--------------------+------------+---------+--------------------+--------------------+-----------+----------------+---------------+------------------+--------------+--------------+--------------+---------------+-------------+--------------+
|      

In [28]:
team_df = spark.createDataFrame(df5)
team_df.show(5)

+-------+-------+--------------------+
|Team_SK|Team_Id|           Team_Name|
+-------+-------+--------------------+
|      0|      1|Kolkata Knight Ri...|
|      1|      2|Royal Challengers...|
|      2|      3| Chennai Super Kings|
|      3|      4|     Kings XI Punjab|
|      4|      5|    Rajasthan Royals|
+-------+-------+--------------------+
only showing top 5 rows


Data Processing

---



In [29]:
ball_by_ball_df.createOrReplaceTempView("ball")
match_df.createOrReplaceTempView("match")
player_df.createOrReplaceTempView("player")
player_match_df.createOrReplaceTempView("player_match")
team_df.createOrReplaceTempView("team")


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
#Problem 1. Top 10 run scorers
top_batsmen = spark.sql("""
SELECT
    p.Player_Name,
    SUM(b.Runs_Scored) AS total_runs
FROM ball b
JOIN player p
    ON b.striker = p.Player_Id
WHERE b.Runs_Scored IS NOT NULL
GROUP BY p.Player_Name
ORDER BY total_runs DESC
LIMIT 10
""")

top_batsmen.show()

top_batsmen.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/top_batsmen")

+--------------+----------+
|   Player_Name|total_runs|
+--------------+----------+
|      SK Raina|      4548|
|       V Kohli|      4413|
|     RG Sharma|      4207|
|     G Gambhir|      4132|
|     DA Warner|      4014|
|    RV Uthappa|      3778|
|      CH Gayle|      3647|
|      S Dhawan|      3561|
|      MS Dhoni|      3560|
|AB de Villiers|      3486|
+--------------+----------+



In [32]:
#Problem 2 . Best Economy bowlers
best_economy = spark.sql("""
SELECT
    p.Player_Name,
    ROUND(SUM(b.Runs_Scored + b.Extra_runs) / (COUNT(b.Ball_id)/6), 2) AS economy
FROM ball b
JOIN player p
    ON b.bowler = p.Player_Id
WHERE b.Runs_Scored IS NOT NULL
AND b.Extra_runs IS NOT NULL
GROUP BY p.Player_Name
HAVING COUNT(b.Ball_id) >= 300
ORDER BY economy ASC
LIMIT 10
""")
best_economy.show()
best_economy.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/best_economy")

+----------------+-------+
|     Player_Name|economy|
+----------------+-------+
|       SP Narine|    6.4|
|        R Ashwin|   6.49|
|        DW Steyn|    6.6|
|        A Kumble|   6.65|
|      GD McGrath|   6.67|
|  M Muralitharan|    6.7|
|     Rashid Khan|   6.73|
|      SL Malinga|   6.75|
|RE van der Merwe|   6.79|
|      DL Vettori|   6.83|
+----------------+-------+



In [43]:
#Probelm 3. Most Sixes hit
most_sixes = spark.sql("""
SELECT
    p.Player_Name,
    COUNT(*) AS total_sixes
FROM ball b
JOIN player p
    ON b.striker = p.Player_Id
WHERE b.Runs_Scored = 6
AND b.striker IS NOT NULL
GROUP BY p.Player_Name
ORDER BY total_sixes DESC
LIMIT 10
""")

most_sixes.show()

most_sixes.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/most_sixes")

+--------------+-----------+
|   Player_Name|total_sixes|
+--------------+-----------+
|      CH Gayle|        266|
|      SK Raina|        174|
|     RG Sharma|        173|
|     DA Warner|        160|
|       V Kohli|        159|
|AB de Villiers|        158|
|      MS Dhoni|        156|
|     YK Pathan|        149|
|    KA Pollard|        148|
|  Yuvraj Singh|        141|
+--------------+-----------+



In [38]:
#Problem 4. Team-wise Winner
team_wins = spark.sql("""
SELECT
    match_winner,
    COUNT(*) AS wins
FROM match
WHERE match_winner IS NOT NULL
AND TRIM(match_winner) != ''
AND LOWER(match_winner) NOT IN ('nan','tie', 'tied', 'no result', 'abandoned')
GROUP BY match_winner
ORDER BY wins DESC
""")

team_wins.show()

team_wins.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/team_wins")

+--------------------+----+
|        match_winner|wins|
+--------------------+----+
|      Mumbai Indians|  91|
| Chennai Super Kings|  79|
|Kolkata Knight Ri...|  77|
|Royal Challengers...|  73|
|     Kings XI Punjab|  70|
|    Rajasthan Royals|  63|
|    Delhi Daredevils|  62|
| Sunrisers Hyderabad|  42|
|     Deccan Chargers|  29|
|Rising Pune Super...|  15|
|       Gujarat Lions|  13|
|       Pune Warriors|  12|
|Kochi Tuskers Kerala|   6|
+--------------------+----+



In [44]:
#Problem 5. Toss Decision Impact
toss_impact = spark.sql("""
SELECT
    Toss_Name AS toss_decision,
    COUNT(*) AS total_matches,
    SUM(CASE
            WHEN Toss_Winner = match_winner THEN 1
            ELSE 0
        END) AS toss_win_and_match_win
FROM match
WHERE match_winner IS NOT NULL
AND Toss_Winner IS NOT NULL
AND Toss_Name IS NOT NULL
AND TRIM(match_winner) != ''
AND TRIM(Toss_Winner) != ''
AND TRIM(Toss_Name) != ''
AND LOWER(TRIM(match_winner)) NOT IN
    ('nan', 'null', 'tie', 'tied', 'no result', 'abandoned')
GROUP BY Toss_Name
ORDER BY total_matches DESC
""")

toss_impact.show()

toss_impact.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/toss_impact")

+-------------+-------------+----------------------+
|toss_decision|total_matches|toss_win_and_match_win|
+-------------+-------------+----------------------+
|        field|          313|                   173|
|          bat|          261|                   118|
|        Field|           48|                    27|
|          Bat|           10|                     6|
+-------------+-------------+----------------------+



In [42]:
#Probelm 6. Best Death-Over Bowlers (Overs 16–20)
death_bowlers = spark.sql("""
SELECT
    p.Player_Name,
    SUM(b.Runs_Scored + b.Extra_runs) AS runs_conceded,
    COUNT(b.Ball_id) AS balls
FROM ball b
JOIN player p
    ON b.Bowler = p.Player_Id
WHERE b.Over_id BETWEEN 16 AND 20
AND b.Runs_Scored IS NOT NULL
AND b.Extra_runs IS NOT NULL
GROUP BY p.Player_Name
HAVING COUNT(b.Ball_id) >= 120
ORDER BY runs_conceded ASC
""")

death_bowlers.show()

death_bowlers.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/death_over_bowlers")


+-----------------+-------------+-----+
|      Player_Name|runs_conceded|balls|
+-----------------+-------------+-----+
|       PJ Cummins|          167|  124|
|    Iqbal Abdulla|          174|  124|
|    Harmeet Singh|          178|  135|
|           S Kaul|          179|  120|
|         SK Raina|          192|  135|
|         M Kartik|          207|  155|
|        SB Jakati|          209|  131|
|Mustafizur Rahman|          212|  168|
|          MS Gony|          213|  124|
|          SW Tait|          216|  159|
|         A Kumble|          217|  174|
|       PJ Sangwan|          222|  139|
|       DL Vettori|          223|  166|
|       AD Mathews|          225|  155|
|         CH Gayle|          229|  156|
|         R Sharma|          230|  156|
|  NM Coulter-Nile|          234|  146|
|         TA Boult|          235|  131|
|    Anureet Singh|          243|  137|
|        S Aravind|          244|  154|
+-----------------+-------------+-----+
only showing top 20 rows


In [40]:
#Problem 7. Player of the Match Awards
player_awards = spark.sql("""
SELECT
    ManOfMach,
    COUNT(*) AS awards
FROM match
GROUP BY ManOfMach
ORDER BY awards DESC
LIMIT 10
""")
player_awards.show()

player_awards.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/drive/MyDrive/IPL_data_analysis/player_of_match_awards")

+--------------+------+
|     ManOfMach|awards|
+--------------+------+
|      CH Gayle|    18|
|     YK Pathan|    16|
|AB de Villiers|    15|
|     DA Warner|    15|
|     RG Sharma|    14|
|      SK Raina|    14|
|     G Gambhir|    13|
|      MS Dhoni|    13|
|     AM Rahane|    12|
|    MEK Hussey|    12|
+--------------+------+

